In [ ]:
# AOI = 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))'
AOI = "MULTIPOLYGON (((-110.49367408906883 39.1005200348037, -110.20673076923077 39.1005200348037, -110.16573886639677 38.41732165423689, -110.43901821862349 38.41732165423689, -110.49367408906883 39.1005200348037)))"
START_DATE = "2022-05-01"
END_DATE = "2022-06-30"

REQUEST_ID = '0'

In [ ]:
import os
import json
import time
import shutil
import rasterio
import rasterio.mask as riomask
import shapely
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt

from utils import crop_raster, merge_tiles, get_tiles

import warnings
warnings.filterwarnings('ignore')

In [ ]:
NAME = "Fields' boundaries"
NB_USER = os.getenv('NB_USER')
# NB_USER = "quantum"
BASE = os.getcwd()
DATA_DIR = f"/home/{NB_USER}/work/jupyter/Landcover"

LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
RESULTS_DIR = os.path.join(DATA_DIR, "results/landcover")

In [ ]:
default_crs = 'EPSG:4326'
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
aoi = gpd.read_file(aoi_filename)

In [ ]:
import urllib.request

sentinel_tiles_path = os.path.join(DATA_DIR, "sentinel2grid.geojson")
# sentinel_tiles_path = "sentinel2grid.geojson"
tile_idx = get_tiles(aoi_filename, sentinel_tiles_path)
year = int(START_DATE.split("-")[0])

if year >= 2022:
    year = 2021
elif year < 2018:
    year = 2018
year

In [ ]:
files = []
data_dir = f"{DATA_DIR}/landcover_dataset"
os.makedirs(data_dir, exist_ok=True)

for tile_i in tile_idx.tileID:
    tile_i = tile_i if len(tile_i) < 4 else tile_i[:3]
    # https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc2018/40S_20180101-20190101.tif
    tile_url = f"https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year-1}/{tile_i}_{year-1}0101-{year}0101.tif"
    path = f"{data_dir}/{tile_i}_{year-1}0101-{year}0101.tif"
    if not os.path.exists(path):
        urllib.request.urlretrieve(tile_url, path)
    files.append(path)
    
files

In [ ]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

In [ ]:
if len(files) > 1:
    raster_path = merge_tiles(
        files, os.path.join(data_dir, "aoi_raster.tif"))
else:
    raster_path = files[0]

In [ ]:
raster_path = crop_raster(raster_path, aoi_filename, raster_path.replace(".tif", "_crop.tif"))

with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

In [ ]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
#     print(label)
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": class_names[name],
            "area": class_area
        })
    else:
        labels.append({
            "color": class_color, 
            "name": name
        })
labels = json.dumps(labels)
labels

In [ ]:
os.remove(aoi_filename)

In [ ]:
# NB_USER = os.getenv('NB_USER')
os.makedirs(f"/home/{NB_USER}/work/results/landcover", exist_ok=True)
tile_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"

In [ ]:
shutil.move(
    raster_path, f"/home/{NB_USER}/work/results/landcover/{tile_name}")